In [1]:
# change tabula to tabula_middle_padding to test middle padding method
from tabula_middle_padding import Tabula 
import pandas as pd
import torch

In [2]:
data = pd.read_csv("Real_Datasets/insurance_original.csv")

In [3]:
#categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
model = Tabula(llm='distilgpt2', experiment_dir = "insurance_training", batch_size=32, epochs=128)

## In following block, it is important to indicate "conditional_col = data.columns[0]". Otherwise, the generation will use target column as conditional column, which will result in error!

In [4]:
model.fit(data, conditional_col = data.columns[0])

/home/svenlag/tabula_test/Tabula/tabula_middle_padding/tabula.py:130: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TabulaTrainer.__init__`. Use `processing_class` instead.
  tabula_trainer = TabulaTrainer(self.model, training_args, train_dataset=tabula_ds, tokenizer=self.tokenizer,
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,2.258800
1000,1.164200
1500,0.684000
2000,0.528000
2500,0.479400
3000,0.459000
3500,0.445600
4000,0.435800
4500,0.427000
5000,0.420600


In [5]:
import torch
torch.save(model.model.state_dict(), "insurance_training/model_128epoch_middle_padding.pt")

In [5]:
torch.save(model, "insurance_training/model_128epoch_middle_padding_FULL.pt")

In [3]:
# Specify weights_only=False to allow full unpickling
model = torch.load(
    "insurance_training/model_128epoch_middle_padding_FULL.pt",
    map_location=torch.device("cuda"),      # or "cpu" as needed
    weights_only=False
)
model.model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [4]:
for i in range(1,31):
    synthetic_data = model.sample(n_samples=1338)
    synthetic_data.to_csv(f"insurance_sd-sets/insurance_128epoch_middle_padding_{i}.csv", index=False)

  0%|          | 0/1338 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
1400it [00:26, 51.94it/s]                          
1400it [00:35, 39.69it/s]                          
1400it [00:43, 32.06it/s]                          
1400it [00:38, 36.76it/s]                          
1400it [00:42, 32.60it/s]                          
1400it [00:39, 35.03it/s]                          
1400it [00:45, 30.98it/s]                          
1400it [00:38, 36.01it/s]                          
1400it [00:45, 30.78it/s]                          
1400it [00:41, 33.39it/s]                          
1400it [00:38, 36.66it/s]                          
1400it [00:31, 44.77it/s]                          
1400it [00:27, 51.27it/s]                          
1400it [00:29, 47.56it/s]                          
140

In [6]:
synthetic_data = model.sample(n_samples=1338)
synthetic_data.to_csv("insurance_128epoch_middle_padding.csv", index=False)

  0%|          | 0/1338 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
1400it [00:14, 98.18it/s]                           


In [7]:
from sdmetrics.reports.single_table import QualityReport

synthetic_data = synthetic_data[data.columns.to_list()]

metadata = {
    "columns": {
        "age": {"sdtype": "numerical"},
        "sex": {"sdtype": "categorical"},
        "bmi": {"sdtype": "numerical"},
        "children": {"sdtype": "numerical"},
        "smoker": {"sdtype": "numerical"},
        "region": {"sdtype": "categorical"},
        "charges": {"sdtype": "numerical"},
    }
}

report = QualityReport()
report.generate(data, synthetic_data, metadata)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 7/7 [00:00<00:00, 624.51it/s]|
Column Shapes Score: 87.56%

(2/2) Evaluating Column Pair Trends: |██████████| 21/21 [00:00<00:00, 338.74it/s]|
Column Pair Trends Score: 72.3%

Overall Score (Average): 79.93%

